# ControlNet Trainer
ControlNet (https://github.com/lllyasviel/ControlNet)<br>
Trainer by Dion Timmer<br>
https://github.com/diontimmer/ControlNet-Trainer


In [1]:
%%capture
%cd /content/
#@title Setup
!mkdir dataset
!mkdir dataset/image
!mkdir dataset/conditioning
!mkdir dataset/prompts
!git clone https://github.com/diontimmer/ControlNet-Trainer.git
%cd ControlNet-Trainer
!mkdir output
!mkdir logs
import json, os
!pip install -r requirements.txt

In [ ]:
#@title Mount Drive (optional)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Dataset converter
converter_conditioning_folder = "/content/dataset/conditioning" #@param {type:"string"}
converter_target_folder = "/content/dataset/image" #@param {type:"string"}
converter_prompt_txts_folder = "/content/dataset/prompts" #@param {type:"string"}
converter_output_json = "/content/dataset/prompt.json" #@param {type:"string"}

def filter_image_files(file_list):
    valid_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".tiff"]
    return [f for f in file_list if os.path.splitext(f)[-1].lower() in valid_extensions]

def filter_text_files(file_list):
    return [f for f in file_list if os.path.splitext(f)[-1].lower() == ".txt"]

conditioning_files = filter_image_files([f for f in os.listdir(converter_conditioning_folder) if os.path.isfile(os.path.join(converter_conditioning_folder, f))])
target_files = filter_image_files([f for f in os.listdir(converter_target_folder) if os.path.isfile(os.path.join(converter_target_folder, f))])
prompt_files = filter_text_files([f for f in os.listdir(converter_prompt_txts_folder) if os.path.isfile(os.path.join(converter_prompt_txts_folder, f))])

print(f'Conditionings: {len(conditioning_files)} | Targets: {len(target_files)} | Prompts: {len(prompt_files)}')

prompt_map = {}
for prompt_file in prompt_files:
    with open(os.path.join(converter_prompt_txts_folder, prompt_file), 'r', encoding='utf-8') as pf:
        prompt_map[prompt_file] = pf.read().strip()

# Open the output file
with open(converter_output_json, 'w') as out_file:
    # Loop over each conditioning file
    for conditioning_file in conditioning_files:
        # Extract the base file name to match with target files and prompts
        base_file_name = conditioning_file.rsplit('.', 1)[0]

        # Raise error if matching target or prompt is not found
        target_files_matched = [target_file for target_file in target_files if target_file.startswith(base_file_name)]
        if not target_files_matched:
            raise ValueError(f"No matching target files found for conditioning file: {conditioning_file}")
        if not base_file_name+'.txt' in prompt_map:
            raise ValueError(f"No matching prompt found for conditioning file: {conditioning_file}")

        # Loop over each target file
        for target_file in target_files_matched:
            # Construct the record
            record = {
                "source": conditioning_file,
                "target": target_file,
                "prompt": prompt_map[base_file_name+'.txt']
            }
            # Write the record to the output file
            out_file.write(json.dumps(record, ensure_ascii=False))
            out_file.write('\n')
print(f'Succesfully wrote to {converter_output_json}')

The dataset converter will format your conditioning + target + prompt pairs into a json needed for your sanity/convenience + the controlnet trainer. You can skip this if you already have a .json file. Every conditioning image needs at least one target image and prompt .txt with the same name. Multiple target images can be loaded for a single conditioning image by differentiating using an underscore _. *ie: 1_0.png 1_1.png 1_2.png etc..*

In [ ]:
#@title Dataset
dataset_conditioning_folder = "/content/dataset/conditioning" #@param {type:"string"}
dataset_target_folder = "/content/dataset/image" #@param {type:"string"}
dataset_captions_json = "/content/dataset/prompt.json" #@param {type:"string"}
print('Folders set!')
print(f'Conditionings: {dataset_conditioning_folder}')
print(f'Targets: {dataset_target_folder}')
print(f'Prompts: {dataset_captions_json}')

In [9]:
#@title Options
project_name = "default" #@param {type:"string"}
run_name = "" #@param {type:"string"}
sd_version = "2.1" #@param ["2.1", "1.5"]
output_dir = "/content/output" #@param {type:"string"}
logging_dir = "/content/logs" #@param {type:"string"}
resume_ckpt = "latest" #@param {type:"string"}

wandb_key = "" #@param {type:"string"}
resolution = 512 #@param {type:"integer"}
batch_size = 3 #@param {type:"integer"}
image_logger_freq = 250 #@param {type:"integer"}
learning_rate = 1e-5 #@param
max_steps = 9000 #@param {type:"integer"}
max_epochs = 10 #@param {type:"integer"}
wipe_older_ckpts = False #@param {type:"boolean"}

save_memory = False #@param {type:"boolean"}
image_logger_disabled = False #@param {type:"boolean"}
save_ckpt_every_n_steps = 250 #@param {type:"integer"}
save_top_k = -1 #@param {type:"integer"}
save_weights_only = False #@param {type:"boolean"}
save_last = False #@param {type:"boolean"}
sd_locked = True #@param {type:"boolean"}
only_mid_control = False #@param {type:"boolean"}
gradient_accumulation_steps = 1

multi_gpu = False #@param {type:"boolean"}

config = {
    "project_name": project_name,
    "run_name": run_name,
    "sd_version": sd_version,
    "output_dir": output_dir,
    "logging_dir": logging_dir,
    "resume_ckpt": resume_ckpt,
    "dataset_conditioning_folder": dataset_conditioning_folder,
    "dataset_target_folder": dataset_target_folder,
    "dataset_captions_json": dataset_captions_json,
    "wandb_key": wandb_key,
    "resolution": resolution,
    "save_memory": save_memory,
    "batch_size": batch_size,
    "image_logger_disabled": image_logger_disabled,
    "image_logger_freq": image_logger_freq,
    "learning_rate": learning_rate,
    "max_steps": max_steps,
    "max_epochs": max_epochs,
    "save_ckpt_every_n_steps": save_ckpt_every_n_steps,
    "save_top_k": save_top_k,
    "save_weights_only": save_weights_only,
    "save_last": save_last,
    "sd_locked": sd_locked,
    "only_mid_control": only_mid_control,
    "gradient_accumulation_steps": gradient_accumulation_steps,
    "wipe_older_ckpts": wipe_older_ckpts,
    "multi_gpu": multi_gpu
}

In [ ]:
#@title Save Config
!mkdir -p configs

config_name = "default_config" #@param {type:"string"}

# Save the dictionary to a json file
with open(f'configs/{config_name}.json', 'w') as f:
    json.dump(config, f, indent=4)

print(f'Config saved as configs/{config_name}.json!')

In [ ]:
#@title Start Training
training_config = "default_config" #@param {type:"string"}
!python train.py configs/{training_config}.json